In [1]:
import pandas as pd 

In [2]:
year = '2016'

In [3]:
df = pd.read_csv('points_against_'+year+'.csv')

In [4]:
# Calculate Team Against Ave/Std
for pos in df.Position.unique():
    for team in df.Team_Against.unique():
        num_weeks = df[(df.Team_Against == team) & (df.Position == pos)].Game_Count.sum()
        if num_weeks != 0:
            ave = df[(df.Team_Against == team) & (df.Position == pos)].Points_Against.astype(float).sum() / num_weeks
        else:
            print(df[(df.Team_Against == team) & (df.Position == pos)])
        std = df[(df.Team_Against == team) & (df.Position == pos)].Points_Against.astype(float).std()
        df.loc[(df.Team_Against == team) & (df.Position == pos), 'Against_Ave'] = ave
        df.loc[(df.Team_Against == team) & (df.Position == pos), 'Against_Std'] = std
        df['Against_16th'] = df['Against_Ave'] - df['Against_Std']
        df['Against_84th'] = df['Against_Ave'] + df['Against_Std']

In [5]:
# Calculate Deviation from average
df['Game_Dev'] = (df.Points_Against.astype(float) - df.Against_Ave.astype(float))/df.Against_Ave.astype(float)

In [6]:
# Construct Summary DF from the 'VS' team perspective
df_vs = df[['VS','Position','Year']]
df_vs = df_vs.drop_duplicates()
df_vs = df_vs[df_vs.VS != 'Bye']

In [7]:
# Calculate Ave/Std (of deviations) for the VS teams
df_vs['VS_Ave'] = 0
df_vs['VS_Std'] = 0
for index, row in df_vs.iterrows():
    df_vs.loc[(df_vs['VS'] == row.VS) & (df_vs.Position == row.Position) & (df_vs.Year == row.Year),'VS_Ave'] = df[(df.VS == row.VS) & (df.Position == row.Position) & (df.Year == row.Year)].Game_Dev.mean()
    df_vs.loc[(df_vs['VS'] == row.VS) & (df_vs.Position == row.Position) & (df_vs.Year == row.Year),'VS_Std'] = df[(df.VS == row.VS) & (df.Position == row.Position) & (df.Year == row.Year)].Game_Dev.std()

In [8]:
# Predict scores in a matchup
def matchup(team_against, vs):
    df_against = df[(df['Team_Against'] == team_against)][['Team_Against','Position','Against_Ave','Against_Std','Against_16th','Against_84th']]
    df_against = df_against.drop_duplicates()
    
    df_matchup = pd.merge(left=df_against, right=df_vs[df_vs['VS']==vs], left_on='Position', right_on='Position')
    
    df_matchup['Weighted_16th'] = (df_matchup['VS_Ave']+1-df_matchup['VS_Std'])*df_matchup['Against_Ave']
    df_matchup['Weighted_Mean'] = (df_matchup['VS_Ave']+1)*df_matchup['Against_Ave']
    df_matchup['Weighted_84th'] = (df_matchup['VS_Ave']+1+df_matchup['VS_Std'])*df_matchup['Against_Ave']
    
    return df_matchup
    

In [9]:
matchup('GB', 'SEA')

,Team_Against,Position,Against_Ave,Against_Std,Against_16th,Against_84th,VS,Year,VS_Ave,VS_Std,Weighted_16th,Weighted_Mean,Weighted_84th
0,GB,QB,17.68875,8.700457,8.988293,26.389207,SEA,2016,0.036160,0.527045,9.005609,18.328371,27.651132
1,GB,RB,16.39375,8.520106,7.873644,24.913856,SEA,2016,-0.149940,0.474193,6.161860,13.935667,21.709474
2,GB,WR,28.93750,14.245040,14.692460,43.182540,SEA,2016,-0.057759,0.453805,14.134103,27.266093,40.398082
3,GB,TE,7.97500,6.173985,1.801015,14.148985,SEA,2016,0.586120,1.055583,4.231040,12.649310,21.067581
4,GB,K,8.00000,3.858718,4.141282,11.858718,SEA,2016,0.068026,0.513082,4.439555,8.544208,12.648862
5,GB,DEF,4.43750,4.837811,-0.400311,9.275311,SEA,2016,0.142139,0.852622,1.284733,5.068241,8.851749


In [10]:
df[(df['VS'] == 'SEA')&(df['Team_Against']=='GB')]

,Unnamed: 0,Game_Count,Points_Against,Position,Team_Against,VS,Week,Year,Home_Away,Against_Ave,Against_Std,Against_16th,Against_84th,Game_Dev
443,0,1,5.5,QB,GB,SEA,14,2016,Home,17.68875,8.700457,8.988293,26.389207,-0.689068
978,0,1,16.0,RB,GB,SEA,14,2016,Home,16.39375,8.520106,7.873644,24.913856,-0.024018
1515,0,1,21.0,WR,GB,SEA,14,2016,Home,28.93750,14.245040,14.692460,43.182540,-0.274298
2072,0,1,2.7,TE,GB,SEA,14,2016,Home,7.97500,6.173985,1.801015,14.148985,-0.661442
2614,0,1,4.0,K,GB,SEA,14,2016,Home,8.00000,3.858718,4.141282,11.858718,-0.500000
3167,0,1,-3.0,DEF,GB,SEA,14,2016,Home,4.43750,4.837811,-0.400311,9.275311,-1.676056
